<a href="https://colab.research.google.com/github/luisam19/course_optimizacion/blob/main/2_Corte_en_AMPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ejemplo de modelo de corte usando AMPL sobre Python

En este Cuaderno resolveremos, nuevamente, nuestro problema de corte usando esta vez AMPL como herramienta de modelación pero en Python. Como referencia de [AMPL](https://ampl.com/) puedes recurrir a su página web. Recapitulemos nuestro problema de corte para ayudar a Muebles El Comejen a planear mejor su producción



# Problema de programación de corte

Conocido en inglés como [*Cutting Stock Problem*](https://neos-guide.org/case-studies/sc/mfg/the-cutting-stock-problem/), este problema clásico de la optimización tiene como objetivo determinar cómo cortar unas piezas grandes de tamaño estándar (láminas, rollos, varillas, etc) en piezas más pequeñas para cumplir con unos pedidos dados de las piezas más pequeñas, de tal manera que se minimice la cantidad de desechos o equivalentemente el número de piezas cortadas. En este [link](https://https://neos-guide.org/content/cutting-stock-problem) tenemos la descripción que nos ofrece NEOS del problema. Ahora veamos un ejemplo práctico y su implementación computacional.

# Un ejemplo:

Liliana, la coordinadora de la planta de producción de Muebles El Comején está planeando la producción de la próxima semana. Con base en la demanda de los muebles, sabe que necesita la siguiente cantidad de piezas:

|   Pieza| Requerimiento |
|-------|----|
| **A** | 13 |
| **B** | 7  |
| **C** | 14	|
| **D** | 3 |
| **E** | 35	|
| **F** | 2 |
| **G** | 17  |

En conjunto con Eduardo, el operario de la máquina de corte, y con base en las dimensiones de las láminas de madera que utilizan y las dimensiones de las piezas a cortar,  Liliana ha ideado 10 patrones de corte diferentes (de los múltiples posibles). El número de piezas obtenidas por cada patrón y los patrones correspondientes se presentan a continuación:


| Pieza/Patron de corte |1|2|3	|4	|5	|6	|7	|8|	9|	10|
|---- |----|---|---	|---|---	|---	|---	|---|---|---|
|A|	1|1|8|0|0|1|2|3|0|1|
|B|	5|0|0|0|0|0|0|2|1|1|
|C|	0|1|0|0|0|1|3|0|0|1|
|D|	1|1|0|0|1|2|0|0|0|1|
|E|	0|0|1|3|0|0|0|0|1|1|
|F|	0|0|1|0|0|1|0|1|0|0|
|G|	0|1|0|0|2|0|0|0|1|0|



#1 Notación

Para formular este modelo iniciamos definiendo la notación necesaria:

**Conjuntos**

> Piezas: P={A, B, C, D, E,F,G}

> Patrones de corte: C={1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

**Parámetros**
> $r_{i}$: Requerimiento de la pieza  $i \in P$   [Unidades]

> $pc_{ij}$: Número de piezas $i \in P$ que se obtienen al cortar una lámina con el patrón $j \in C$  [Unidades]

**Variables de decisión**

> $x_{j}$: Variable entera, cantidad de láminas que se cortarán usando el patrón $j \in C$.

#2 Verbalización
Ahora, expresamos en palabras el modelo que queremos constuir, declarando explícitamente su <font color='blue'>función objetivo</font>,<font color='red'> restricciones</font> y <font color='green'>decisiones</font>:

<font color='green'>Decidir cuántas láminas se deben cortar con cada patrón de corte, </font> <font color='red'> cumpliendo el requerimiento mínimo de cada pieza (demanda) </font> <font color='blue'>, para minimizar el número total de láminas cortadas.</font>

#3 Formulación

Ahora, usando la notación de 1. y la verbalización de 2., tenemos el siguiente modelo de optimización que permitirá planear óptimamente la asignación de los casos en el bufete de abogados.

>$\text{min}\ LaminasTotales = \sum_{j \in C}x_{j} $    (<font color='blue'> Total de láminas cortadas  </font>)

$sujeto \, a:$

>$\sum_{j \in C}pc_{ij} \, x_{j}\geq r_{i}, \forall i \in P \quad$(<font color='red'> Requerimiento mínimo de cada pieza (demanda)  </font>)

> $x_{j} \in  \mathbb Z^+ \quad \quad \quad \quad \forall j \in C   \quad \quad$           (<font color='green'>Dominio de las variables - enteras- </font>)


#4 Implementación computacional

Y ahora si, la implementación computacional de este modelo usando para ello AMPL sobre Python

In [ ]:
#Instalemos primero los paquetes que necesitamos
!pip install -q amplpy pandas
from amplpy import AMPL, tools
ampl = tools.ampl_notebook(
    modules=["highs", "cbc", "gurobi"], # Los optimizadores que vamos a usar
    license_uuid="default") # license to use (Aqui hay que poner su licencia :-;


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 27.7 MB/s eta 0:00:00
Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://colab.ampl.com).


In [ ]:
#Ahora vamos a utilizar nuestro modelo de AMPL despues de resetear el ambiente
%%ampl_eval
reset;

#Conjuntos
set Piezas;                	#Conjunto de piezas a fabricas
set Patrones;				#Conjunto de patrones de corte disponibles
#Parametros
param r{Piezas}>=0;         # Requerimiento de cada pieza
param pc {Piezas,Patrones}>=0; #Numero de piezas cortadas en cada patron
#Variables
var x{j in Patrones}>=0, integer;    #numero de veces que se cortara cada patron

#Funcion objetivo
minimize LaminasTotales: sum{j in Patrones}x[j];

#Restricciones
subject to Demanda{i in Piezas}:
sum{j in Patrones}pc[i,j]*x[j]>=r[i];


# Ahora los datos

In [ ]:
#Parametros cargados directamente
ampl.set["Piezas"] = ["A", "B", "C", "D", "E", "F", "G"]
ampl.set["Patrones"] = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] #En Python contamos desde 0 ! Acuerdate de esto
ampl.param["r"] = {"A": 13, "B": 7, "C": 14, "D": 3, "E":  35, "F":2, "G": 17 }

In [ ]:
#Ahora unos desde Pandas
import pandas as pd
pc_df = pd.DataFrame({
        "A": [1,1,8,0,0,1,2,3,0,1],
        "B": [5,0,0,0,0,0,0,2,1,1],
        "C": [0,1,0,0,0,1,3,0,0,1],
        "D": [1,1,0,0,1,2,0,0,0,1],
        "E": [0,0,1,3,0,0,0,0,1,1],
        "F": [0,0,0,0,0,1,0,1,0,0],
        "G": [0,1,0,0,2,0,0,0,1,0]
    })
print(pc_df)

ampl.param["pc"] = pc_df.unstack()

   A  B  C  D  E  F  G
0  1  5  0  1  0  0  0
1  1  0  1  1  0  0  1
2  8  0  0  0  1  0  0
3  0  0  0  0  3  0  0
4  0  0  0  1  0  0  2
5  1  0  1  2  0  1  0
6  2  0  3  0  0  0  0
7  3  2  0  0  0  1  0
8  0  1  0  0  1  0  1
9  1  1  1  1  1  0  0


In [ ]:
#Y finalmente resolvamos usando un software gratuito nuevo: HIGHS y pongamos la respuesta en un dataframe nuevamente

ampl.option["solver"] = "highs"
ampl.solve()
print("El valor optimo de la funcion objetivo es :",ampl.get_data("LaminasTotales"))
print("Las laminas se cortan según los siguentes patrones (Patrón- Número de cortes)")

ampl.get_data("x").to_pandas()

